# Prepare CNN Data

## Setup

In [ ]:
# USE ACCOUNT: ramarty@email.wm.edu
import ee
ee.Authenticate()
ee.Initialize()

In [26]:
import numpy as np
#import geetools
#from geetools import ui, cloud_mask
import os, datetime
import pandas as pd
import itertools
import tensorflow as tf
import config as cf
import ee_utils as utils
import eeconvert

#cloud_mask_landsatSR = cloud_mask.landsatSR()
#cloud_mask_sentinel2 = cloud_mask.sentinel2()

## Parameters

In [27]:
KERNEL_SIZE = 500 #224
SURVEY_NAME = 'DHS'
SATELLITE = 's2'
SKIP_IF_SCRAPED = True
CHUNK_SIZE = 2 # Number of observtaions to scrape in GEE at any given time
DROPBOX_DIR = cf.DROPBOX_DIRECTORY
GOOGLEDRIVE_DIR = cf.GOOGLEDRIVE_DIRECTORY

In [28]:
# Directory to store tfrecords
out_path = os.path.join(GOOGLEDRIVE_DIR, 
            'Data', 
            SURVEY_NAME, 
            'FinalData',
            'Individual Datasets',
            'cnn_' + SATELLITE,
            'tfrecords')

## Load Data

In [29]:
# Relies on VIIRs Values
survey_df = pd.read_csv(os.path.join(DROPBOX_DIR, 'Data', SURVEY_NAME, 'FinalData', 'Individual Datasets', 'survey_socioeconomic.csv'))

viirs_df = pd.read_csv(os.path.join(DROPBOX_DIR, 'Data', SURVEY_NAME, 'FinalData', 'Individual Datasets', 'survey_viirs_5km.csv'))
viirs_df.viirs_avg_rad = np.log(viirs_df.viirs_avg_rad + 1)

survey_df = survey_df.merge(viirs_df, on = 'uid')

In [30]:
# Sentinel doesn't capture Lakshadweep (island off coast of mainland India)
survey_df = survey_df[survey_df['GID_2'] != "IND.18.1_1"]

# List of TF Records
tf_record_list = list(np.unique(survey_df.tfrecord_name))

In [31]:
# Remove if issues extracting
#survey_df = survey_df[survey_df['uid'] != "IA201400180079"]
#survey_df = survey_df[survey_df['uid'] != "IA201400180052"]
#survey_df = survey_df[survey_df['uid'] != "IA201400180112"]
#survey_df = survey_df[survey_df['uid'] != "IA201400180081"]
#survey_df = survey_df[survey_df['uid'] != "IA201400180011"]
#survey_df = survey_df[survey_df['uid'] != "IA201400180048"]
#survey_df = survey_df[survey_df['uid'] != "IA201400180058"]
#survey_df = survey_df[survey_df['uid'] != "IA201400180028"]
#survey_df = survey_df[survey_df['uid'] != "IA201400180072"]
#survey_df = survey_df[survey_df['uid'] != "IA201400180047"]
#survey_df = survey_df[survey_df['uid'] != "IA201400180012"]
#survey_df = survey_df[survey_df['uid'] != "IA201400180040"]
#survey_df = survey_df[survey_df['uid'] != "IA201400180055"]
#survey_df = survey_df[survey_df['uid'] != "IA201400180140"]
#survey_df = survey_df[survey_df['uid'] != "IA201400180030"]
#survey_df = survey_df[survey_df['uid'] != "IA201400180104"]
#survey_df = survey_df[survey_df['uid'] != "IA201400180123"]
#survey_df = survey_df[survey_df['uid'] != "IA201400180062"]
#survey_df = survey_df[survey_df['uid'] != "IA201400180080"]
#survey_df = survey_df[survey_df['uid'] != "IA201400180050"]
#survey_df = survey_df[survey_df['uid'] != "IA201400180116"]

In [32]:
# If skip already scraped, remove existing tfrecords from tf_record_list
if SKIP_IF_SCRAPED:
    tf_records_exist = os.listdir(out_path)
    tf_record_list = [x for x in tf_record_list if x not in tf_records_exist]

In [33]:
tf_record_list.reverse()

In [ ]:
### Loop through all tfrecords
for tfr_i in tf_record_list:
    
    survey_df_yeari = survey_df[survey_df['tfrecord_name'] == tfr_i]
    year_i = survey_df_yeari['year'].iloc[0]

    ### Loop through chunks within tfrecord (can only pull so much data from GEE at a time)
    survey_df_yeari['chunk_id'] = utils.chunk_ids(survey_df_yeari.shape[0], CHUNK_SIZE)

    print("Putting " + str(survey_df_yeari.shape[0]) + " observations into " + tfr_i)
    
    proto_examples_all = []
    for chunk_i in list(np.unique(survey_df_yeari.chunk_id)):
        
        print("Observation: " + str(len(proto_examples_all)) + "/" + str(survey_df_yeari.shape[0]))
        
        survey_df_yeari_chunki = survey_df_yeari[survey_df_yeari['chunk_id'] == chunk_i]
      
        proto_examples_i = utils.prep_cnn_np(survey_df_yeari_chunki, SATELLITE, KERNEL_SIZE, year_i)
        proto_examples_all.extend(proto_examples_i)

    ### Save data as tf record
    out_path_i = os.path.join(out_path, tfr_i)
    print(out_path_i)
    with tf.io.TFRecordWriter(out_path_i) as writer:
        for tf_example in proto_examples_all:
            writer.write(tf_example.SerializeToString())

/Users/robmarty/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Putting 125 observations into TL_5_1.tfrecord
Observation: 0/125
Observation: 2/125
Observation: 4/125
Observation: 6/125
Observation: 8/125
Observation: 10/125
Observation: 12/125
Observation: 14/125
Observation: 16/125
Observation: 18/125
Observation: 20/125
Observation: 22/125
Observation: 24/125
Observation: 26/125
Observation: 28/125
Observation: 30/125
Observation: 32/125
Observation: 34/125
Observation: 36/125
Observation: 38/125
Observation: 40/125
Observation: 42/125
Observation: 44/125
Observation: 46/125
Observation: 48/125
Observation: 50/125
Observation: 52/125
Observation: 54/125
Observation: 56/125
Observation: 58/125
Observation: 60/125
Observation: 62/125
Observation: 64/125
Observation: 66/125
Observation: 68/125
Observation: 70/125
Observation: 72/125
Observation: 74/125
Observation: 76/125
Observation: 78/125
Observation: 80/125
Observation: 82/125
Observation: 84/125
Observation: 86/125
Observation: 88/125
Observation: 90/125
Observation: 92/125
Observation: 94/125

In [ ]:
for row_i in range(0,10):
    print(row_i)
    proto_examples_i = utils.prep_cnn_np(survey_df_yeari_chunki.iloc[[row_i]], SATELLITE, KERNEL_SIZE, year_i)

In [ ]:
survey_df_yeari_chunki.iloc[[1]]